In [1]:
import os

ROOT_DATA = os.environ.get('LARD_DATA_ROOT_PATH')
ROOT_PROJECT = os.environ.get('LARD_PROJECT_ROOT_PATH')

print(ROOT_DATA)
print(ROOT_PROJECT)


/home/yeli/workspace/lard/lard-dataset
/home/yeli/workspace/lard/lard-detection


# 查看已有数据集

In [2]:
import fiftyone as fo
all_datasets = fo.list_datasets()
print(all_datasets)

['lard_test_real_edge', 'lard_test_real_nominal', 'lard_test_synth', 'lard_train', 'lard_train_all', 'lard_val']


# 打开web界面查看结果

In [4]:
import fiftyone as fo
dataset = fo.load_dataset("lard_test_synth")
print(dataset.get_field_schema().keys())
session = fo.launch_app(dataset, height=700, auto=False)
# session.show()

odict_keys(['id', 'filepath', 'tags', 'metadata', 'ground_truth_detections', 'predictions_yolov8n'])
Session launched. Run `session.show()` to open the App in a cell output.


In [7]:
session.close()

# 删除全部预测

In [8]:
from src.utils.database import delete_all_predictions
    
for name in all_datasets:
    dataset = fo.load_dataset(name)
    delete_all_predictions(dataset)


删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []


: 

# 评估各数据上各模型性能

In [ ]:
from src.utils.database import eval_detections_in_database

all_results = eval_detections_in_database('yolov8n-p2_train_val')

# todo: 写一个函数，自动处理各数据集/各模型 进行评估

In [ ]:
for datatype, results in all_results.items():
    plot_pr = results.plot_pr_curves()
    display(plot_pr)

In [ ]:
for datatype, results in all_results.items():
    plot_mat = results.plot_confusion_matrix()
    display(plot_mat)
